## Ising Model GAN 

In [ ]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, "..")
from input_pipeline import dataset_pipeline
import time
from IPython import display
from model import make_discriminator_model, make_generator_model, generator_loss, discriminator_loss

## Load data

In [ ]:
batch_size = 64

train_path = '../../GetData/Python/Data/Trainset'
train_ds = dataset_pipeline(train_path, flatten=False, batch_size=batch_size)

test_path = '../../GetData/Python/Data/Testset'
test_ds = dataset_pipeline(test_path, flatten=False, batch_size=batch_size)

## Model

In [ ]:
generator = make_generator_model()
discriminator = make_discriminator_model()

## Setup training

In [ ]:
noise_dim = 100
num_examples_to_generate = 16
# We'll re-use this random vector used to seed the generator so
# it will be easier to see the improvement over time.
random_vector_for_generation = tf.random.normal([num_examples_to_generate,
                                                 noise_dim])

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
def train_step(images, gen_loss_log, disc_loss_log):
      noise = tf.random.normal([batch_size, noise_dim])
      
      with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
      
        real_output = discriminator(images, training=True)
        generated_output = discriminator(generated_images, training=True)
        gen_loss = generator_loss(generated_output)
        gen_loss_log.append(gen_loss)
        disc_loss = discriminator_loss(real_output, generated_output)
        disc_loss_log.append(disc_loss)
        
      gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
      gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
      
      generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
      discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
def generate_and_save_images(model, epoch, test_input):
  predictions = tf.round(model(test_input, training=False))

  fig = plt.figure(figsize=(4,4))
  
  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(predictions[i, :, :, 0])
      plt.axis('off')
        
  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

In [ ]:
def train(dataset, epochs, gen_loss_log, disc_loss_log):  
  
  for epoch in range(epochs):
    start = time.time()

    for images in tqdm(dataset): train_step(images, gen_loss_log, disc_loss_log)

    display.clear_output(wait=True) 
    generate_and_save_images(
      generator,
      epoch + 1,
      random_vector_for_generation
    ) 
    print (f"Time taken for epoch {epoch} is {time.time()- start} sec")
  # saving (checkpoint) the model every 15 epochs
  #  if (epoch + 1) % 15 == 0:
  #    checkpoint.save(file_prefix = checkpoint_prefix)

## Train the GAN!

In [ ]:
EPOCHS=5
gen_loss_log=[]
disc_loss_log=[]

In [ ]:
%%time
train(test_ds, 1 ,gen_loss_log,disc_loss_log)

## Plot the loss of the generator and discriminator

In [ ]:
plt.plot(np.asarray(gen_loss_log))
plt.plot(np.asarray(disc_loss_log))

In [ ]:
predictions = generator(random_vector_for_generation, training=False)

In [ ]:
predictions